# Cognica를 이용한 IMDB 데이터 벡터 검색하기

벡터 검색은 문자 데이터를 임베딩 벡터로 변환하고 이를 이용하여 입력된 질의와 유사한 벡터를 찾아 검색하는 방법입니다. 벡터 임베딩은 입력 데이터를 특정 공간에 사상하는 함수로 하나 이상의 입력 데이터가 의미상, 문맥상 유사하다면 근접한 공간에 할당되게 됩니다. 따라서 벡터의 거리값이 가까우면 실제로 의미상, 문맥상 의미가 유사함을 추정할 수 있게 됩니다. 입력 문자 데이터를 벡터로 변환하는 벡터 임베딩 모델은 입력 데이터가 하나 이상의 문장으로 학습되는 경우가 많아 단순히 단어, 구문을 넘어서 자연어로 입력되더라도 입력된 문장의 의미가 크게 손실되지 않고 변환할 수 있습니다. 따라서 입력된 질의가 자연어이고 의미를 살려 검색하고자 할 때 활용할 수 있습니다.

여기서는 IMDB 데이터를 전문 검색을 위한 색인과 벡터 검색 색인을 생성하고 검색하는 방법에 대해서 다룹니다.

## 준비 사항

### 의존성 설치

이 예제는 Cognica가 설치되어 있고 서버가 동작하고 있음을 가정합니다. 설치 방법은 [Cognica 설치하기](https://docs.cognica.io/install)를 참고하여 설치할 수 있습니다.

동작에 필요한 라이브러리를 설치합니다.

In [1]:
!pip install -U pip
!pip install cognica

### 데이터 준비하기

데이터는 IMDB 데이터를 사용하며 이 파일(`imdb_top1000.csv`)은 예제에 미리 준비되어 있습니다. 이 데이터는 [IMDB Movies Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)에서 데이터 대한 내용을 확인할 수 있습니다.

데이터를 입수하면 늘 그렇듯 데이터를 살펴보고 의도에 맞게 가공해야 합니다. 데이터의 칼럼은 하나 이상의 타입을 가질 수 있고 이는 데이터를 검색하고 조회하는데 방해가 될 수 있습니다. 특히 대부분 데이터를 다루는 라이브러리에서 NA(Not a Number) float는 타입이고 이는 불편을 유발하는 경우가 많습니다. 따라서 향후 검색을 편리하도록 하기 위해 몇몇 필드는 타입이 일치하도록 가공합니다. 아래 코드에서는 `read_csv`로 데이터를 로딩하는 과정에서 `converter`를 통해 변환하고 있습니다.

Cognica의 특성상 각 칼럼이 같은 타입을 가지도록 강제하지 않습니다. 이 내용은 [Cognica가 데이터를 다루는 특징](https://docs.cognica.io/user-guide/documentdb/data#cognica%EA%B0%80-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC-%EB%8B%A4%EB%A3%A8%EB%8A%94-%ED%8A%B9%EC%A7%95)에서 참고할 수 있습니다.

In [2]:
import pandas as pd

converters = {
    "Gross": lambda x: int(x.replace(",", "")) if x else 0,
    "Certificate": lambda x: x if x else "",
    "Released_Year": lambda x: 0 if x == "PG" else int(x),
}
df = pd.read_csv("imdb_top_1000.csv", converters=converters)
display(df.dtypes)
df.head()

Poster_Link       object
Series_Title      object
Released_Year      int64
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross              int64
dtype: object

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000


이제 데이터가 정상적으로 준비되었고 이 데이터를 Cognica에 입력합니다.

## Cognica에 데이터 입력

이제 `create_collection`을 통해 `imdb3`이라는 컬렉션을 생성합니다. 색인 생성을 위한 `indexes`를 정의합니다.

하나씩 살펴보면 다음과 같습니다. `index`를 primary key로 정의합니다.

```python
{"fields": ["index"], "unique": True, "index_type": "kPrimaryKey"},
```

다음은 전문검색을 위한 색인입니다. 색인의 이름을 `sk_fts`로 정의하고 전문 검색에 사용할 필드를 `fields`로 정의합니다.

```python
"name": "sk_fts",
"fields": ["index", "IMDB_Rating", "Series_Title", "Director", "Overview", "OverviewEmbed"],
"index_type": "kFullTextSearchIndex",
```

다음은 각 필드에 대한 분석기를 정의합니다. 특별히 `IMDB_Rating`은 float 타입이기 때문에 `Float64Analyzer`를 사용하고 나머지는 일반적인 토큰 분리를 처리하는 `StandardAnalyzer`로 정의하고 있습니다. 여기에 추가로 `OverviewEmbed` 필드를 추가하고 벡터 데이터를 위한 스키마를 정의합니다. 벡터 데이터를 위한 색인을 생성할 때 주의할 점은 벡터 데이터의 차원(`dims`)을 일치시켜야 합니다. 

자세한 내용은 [전문검색 색인 스키마](https://docs.cognica.io/user-guide/documentdb/collection/index-fts)를 통해 참고할 수 있습니다.

```python
"options": {
    "index": {
        "analyzer": {"type": "KeywordAnalyzer"},
        "index_options": "doc_freqs",
    },
    "IMDB_Rating": {
        "analyzer": {"type": "Float64Analyzer"},
        "index_options": "doc_freqs",
    },
    "Series_Title": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
    "Director": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
    "Overview": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
    "OverviewEmbed": {
        "analyzer": {
            "type": "DenseVectorAnalyzer",
            "options": {
                "index_type": "HNSW",
                "dims": 768,
                "m": 64,
                "ef_construction": 200,
                "ef_search": 32,
                "metric": "inner_product",
                "normalize": True,
                "shards": 1
            }
        },
        "index_options": "doc_freqs"
    }
},
```

In [3]:
from cognica import Channel, DocumentDB

channel = Channel("localhost", 10080)
doc_db = DocumentDB(channel)

indexes = [
    {"fields": ["index"], "unique": True, "index_type": "kPrimaryKey"},
    {
        "name": "sk_fts",
        "fields": ["index", "IMDB_Rating", "Series_Title", "Director", "Overview", "OverviewEmbed"],
        "unique": False,
        "index_type": "kFullTextSearchIndex",
        "options": {
            "index": {
                "analyzer": {"type": "KeywordAnalyzer"},
                "index_options": "doc_freqs",
            },
            "IMDB_Rating": {
                "analyzer": {"type": "Float64Analyzer"},
                "index_options": "doc_freqs",
            },
            "Series_Title": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
            "Director": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
            "Overview": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
            "OverviewEmbed": {
                "analyzer": {
                    "type": "DenseVectorAnalyzer",
                    "options": {
                        "index_type": "HNSW",
                        "dims": 384,
                        "m": 64,
                        "ef_construction": 200,
                        "ef_search": 32,
                        "metric": "inner_product",
                        "normalize": True,
                        "shards": 1
                    }
                },
                "index_options": "doc_freqs"
            }
        },
    },
]

if "imdb3" in doc_db.list_collections():
    doc_db.drop_collection("imdb3")
doc_db.create_collection("imdb3", indexes=indexes)

이제 색인이 준비되었고 데이터를 입력합니다.

Cognica에는 모델 서빙 기능이 내장되어 있고 이를 이용하여 임베딩 벡터로 변환할 수 있습니다. 모델 동작을 위해서 [서버 설정](https://docs.cognica.io/operation-guide/setting#ml-model-serving)이 필요 합니다.

```python
from cognica import SentenceTransformerEncoder

encoder = SentenceTransformerEncoder(
    channel, "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)
```

IMDB 데이터 중 `Overview`에 대해서 임베딩 벡터로 변환합니다.

```python
overviews = df.Overview.to_list()
df["OverviewEmbed"] = encoder.encode(overviews).tolist()
```

In [4]:
from cognica import SentenceTransformerEncoder

encoder = SentenceTransformerEncoder(
    channel, "intfloat/multilingual-e5-small"
)

overviews = df.Overview.to_list()
df["OverviewEmbed"] = encoder.encode(overviews).tolist()
data = df.reset_index().to_dict(orient="records")
doc_db.insert("imdb3", data)

## 검색하기

이제 검색할 준비가 되었습니다. 우선 전문 검색을 통해 `Overview`에서 `crime`이 출현하는 문서를 찾아보도록 하겠습니다. 전문 검색은 `$search` 연산자를 사용하고 `query`에 검색어를 입력합니다. 쿼리에 입력 가능한 검색어 문법은 `AND`, `OR`와 같은 불리언 연산이나 와일드 카드, 근접 검색을 사용할 수 있습니다. 자세한 내용은 [전문 검색](https://docs.cognica.io/user-guide/documentdb/search/fts)에서 살펴볼 수 있습니다.
그리고 `$project`로 전달받을 칼럼과 `$hint`를 통해 `sk_fts` 색인이 선택되도록 유도합니다.

In [5]:
query = "crime"
project = [
        "Series_Title",
        "Director",
        "Overview",
        "IMDB_Rating",
        {"_meta": "$unnest"}
]
search_query = {
    "$search": {
        "query": f"Overview:({query})",
    },
    "$project": project,
    "$hint": "sk_fts"
}

doc_db.find("imdb3", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Jules Dassin,8.2,"Four men plan a technically perfect crime, but...",Du rififi chez les hommes,347,1.907696
1,François Truffaut,8.1,"A young boy, left without attention, delves in...",Les quatre cents coups,459,1.907696
2,Francis Ford Coppola,9.2,An organized crime dynasty's aging patriarch t...,The Godfather,14,1.789835
3,Christopher Nolan,8.2,"After training with his mentor, Batman begins ...",Batman Begins,316,1.789835
4,Stanley Kramer,8.2,"In 1948, an American court in occupied Germany...",Judgment at Nuremberg,343,1.789835
5,Jeethu Joseph,8.3,A man goes to extreme lengths to save his fami...,Drishyam,723,1.789835
6,Aniruddha Roy Chowdhury,8.1,When three young women are implicated in a cri...,Pink,357,1.736202
7,Francis Ford Coppola,7.6,"Follows Michael Corleone, now in his 60s, as h...",The Godfather: Part III,900,1.736202
8,Nadine Labaki,8.4,While serving a five-year sentence for a viole...,Capharnaüm,66,1.685690
9,Edgar Wright,7.6,After being coerced into working for a crime b...,Baby Driver,606,1.685690


이제 `infraction`을 검색합니다. `crime`와 유의어임에도 결과가 없음을 확인할 수 있습니다. 

In [6]:
query = "infraction"
embed = encoder.encode([query])[0]
query_embed = ",".join(map(str, embed.tolist()))
search_query = {
    "$search": {
        "query": f"Overview:({query})",
    },
    "$project": project,
    "$hint": "sk_fts"
}
doc_db.find("imdb3", search_query).head(10)

""


이번에는 `Overview`를 임베딩하여 생성한 `OverviewEmbed`를 이용하여 검색합니다. 이 때 질의에 입력되는 `query`는 `query_embed`로 변환되어 입력됨을 유의해야 합니다.
검색 결과는 다음과 같이 하나 이상이 결과를 확인할 수 있고 실제로 `infraction`이 출현하지 않더라도 검색됨을 확인할 수 있습니다.

벡터 검색은 전문 검색보다 일반적으로는 좋은 결과를 보이지만 비용적인 측면이나 성능면에서 항상 그렇지는 않습니다. 따라서 필요에 따라 적절한 검색 방법을 선택하는 것이 필요합니다.

In [7]:
query = "infraction"
embed = encoder.encode([query])[0]
query_embed = ",".join(map(str, embed.tolist()))
search_query = {
    "$search": {
        "query": f"OverviewEmbed:([{query_embed}])",
    },
    "$project": project,
    "$hint": "sk_fts"
}
doc_db.find("imdb3", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Orson Welles,8.0,"A stark, perverse story of murder, kidnapping,...",Touch of Evil,292,0.816783
1,Sidney Lumet,9.0,A jury holdout attempts to prevent a miscarria...,12 Angry Men,17,0.814285
2,James Foley,7.7,An examination of the machinations behind the ...,Glengarry Glen Ross,987,0.810963
3,Joel Coen,8.1,Jerry Lundegaard's inept crime falls apart due...,Fargo,415,0.809659
4,Neeraj Pandey,8.1,A retiring police officer reminisces about the...,A Wednesday,385,0.809405
5,Oriol Paulo,8.1,A successful entrepreneur accused of murder an...,Contratiempo,359,0.807425
6,Andrew Lau,8.0,A story between a mole in the police departmen...,Mou gaan dou,684,0.807411
7,Steven Spielberg,7.6,In a future where a special police unit is abl...,Minority Report,877,0.807057
8,Robert Aldrich,8.1,A former child star torments her paraplegic si...,What Ever Happened to Baby Jane?,453,0.806639
9,Jules Dassin,8.2,"Four men plan a technically perfect crime, but...",Du rififi chez les hommes,347,0.806185


이번에는 단어가 아니라 자연어를 입력합니다. 위에서 설명한 바와 같이 임베딩 모델의 입력 데이터는 단어에 국한되지 않습니다. 검색 대상 데이터가 많지 않지만 적절한 결과가 검색됨을 확인할 수 있습니다.

In [8]:
query = "A movie about chasing criminals"
embed = encoder.encode([query])[0]
query_embed = ",".join(map(str, embed.tolist()))
search_query = {
    "$search": {
        "query": f"OverviewEmbed:([{query_embed}])",
    },
    "$project": project,
    "$hint": "sk_fts"
}
doc_db.find("imdb3", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,David Ayer,7.6,"Shot documentary-style, this film follows the ...",End of Watch,543,0.860574
1,Andrew Niccol,7.6,An arms dealer confronts the morality of his w...,Lord of War,857,0.857561
2,Orson Welles,8.0,"A stark, perverse story of murder, kidnapping,...",Touch of Evil,292,0.846187
3,Buster Keaton,8.2,"A film projectionist longs to be a detective, ...",Sherlock Jr.,355,0.845965
4,Richard Kelly,8.0,"After narrowly escaping a bizarre accident, a ...",Donnie Darko,312,0.845729
5,Kar-Wai Wong,7.7,This Hong Kong-set crime drama follows the liv...,Do lok tin si,978,0.844321
6,Zoya Akhtar,8.0,A coming-of-age story based on the lives of st...,Gully Boy,643,0.844167
7,Billy Wilder,8.4,A screenwriter develops a dangerous relationsh...,Sunset Blvd.,718,0.841499
8,Jean-Luc Godard,7.8,A small-time thief steals a car and impulsivel...,À bout de souffle,787,0.840873
9,Jonathan Demme,8.6,A young F.B.I. cadet must receive the help of ...,The Silence of the Lambs,41,0.839872


다음은 전문 검색과 벡터 검색을 혼합한 하이브리드 검색에 대한 예시입니다. 방법은 일반적인 전문 검색과 유사하게 불리언 연산을 통해 두 검색 방식을 병합하고 `^0.2`와 같이 [부스팅](https://docs.cognica.io/user-guide/documentdb/search/fts.ko#boosting)을 통해 두 검색 방법 간의 가중치를 조절합니다.

두 검색 방법은 서로 장단점이 있으며 아래와 같이 조절하여 최적의 결괏값을 탐색할 수 있습니다.

In [9]:
query = "crime"
embed = encoder.encode([query])[0]
query_embed = ",".join(map(str, embed.tolist()))
search_query = {
    "$search": {
        "query": f"(Overview:({query}))^0.2 AND (OverviewEmbed:[{query_embed}])^20",
    },
    "$project": project,
    "$hint": "sk_fts"
}
doc_db.find("imdb3", search_query).head(10)


,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Jules Dassin,8.2,"Four men plan a technically perfect crime, but...",Du rififi chez les hommes,347,16.914345
1,Nadine Labaki,8.4,While serving a five-year sentence for a viole...,Capharnaüm,66,16.602993
2,Joel Coen,8.1,Jerry Lundegaard's inept crime falls apart due...,Fargo,415,16.590494
3,Jeethu Joseph,8.3,A man goes to extreme lengths to save his fami...,Drishyam,723,16.483292
4,Nishikant Kamat,8.2,Desperate measures are taken by a man who trie...,Drishyam,696,16.469566
5,François Truffaut,8.1,"A young boy, left without attention, delves in...",Les quatre cents coups,459,16.465488
6,Francis Ford Coppola,9.2,An organized crime dynasty's aging patriarch t...,The Godfather,14,16.450748
7,Steven Spielberg,7.6,In a future where a special police unit is abl...,Minority Report,877,16.427301
8,Kar-Wai Wong,7.7,This Hong Kong-set crime drama follows the liv...,Do lok tin si,978,16.408073
9,Alfred Hitchcock,8.0,Two men attempt to prove they committed the pe...,Rope,304,16.393190
